In [1]:
import re
import os
from langchain.vectorstores.neo4j_vector import Neo4jVector
from langchain.document_loaders import WikipediaLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import (
    CharacterTextSplitter,
    RecursiveCharacterTextSplitter,
)
from langchain.graphs import Neo4jGraph
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from langchain.text_splitter import TokenTextSplitter

from secret import *
import chromadb

os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [2]:
graph = Neo4jGraph(url="bolt://localhost:7687", username="neo4j", password="test1234")
graph.query(
    """
MATCH (n)
DETACH DELETE n
"""
)
graph.refresh_schema()
print(graph.schema)


        Node properties are the following:
        []
        Relationship properties are the following:
        []
        The relationships are the following:
        []
        


In [3]:
raw_documents = WikipediaLoader(query="Atelectasis").load()
text_splitter = TokenTextSplitter(chunk_size=2048, chunk_overlap=24)
documents = text_splitter.split_documents([raw_documents[0]]) # should I only take the first? which is the most relevant?

In [4]:
db = Neo4jVector.from_documents(
    documents, OpenAIEmbeddings(), url=NOE4J_URL, username="neo4j", password="test1234"
)

retriever = db.as_retriever()

In [5]:
from langchain.chains import RetrievalQAWithSourcesChain
from langchain.chat_models import ChatOpenAI

In [6]:
chain = RetrievalQAWithSourcesChain.from_chain_type(
    ChatOpenAI(temperature=0, model="gpt-3.5-turbo-16k"), chain_type="stuff", retriever=retriever
)

In [7]:
chain(
    {"question": "What is Atelectasis?"},
    return_only_outputs=True,
)

{'answer': 'Atelectasis is the collapse or closure of a lung resulting in reduced or absent gas exchange. It is usually unilateral, affecting part or all of one lung. It is a condition where the alveoli are deflated down to little or no volume. It is often called a collapsed lung, although that term may also refer to pneumothorax. It is a very common finding in chest X-rays and other radiological studies, and may be caused by normal exhalation or by various medical conditions. The most common cause is post-surgical atelectasis, characterized by restricted breathing after abdominal surgery. Other causes include pulmonary tuberculosis, blockage of a bronchiole or bronchus, poor surfactant spreading during inspiration, and relaxation atelectasis. Diagnosis is generally done through chest X-ray, and additional tests such as CT scan or bronchoscopy may be necessary. \n',
 'sources': 'https://en.wikipedia.org/wiki/Atelectasis'}

In [8]:
chain(
    {"question": "What are the symptoms of Atelectasis?"},
    return_only_outputs=True,
)

{'answer': 'The symptoms of Atelectasis may include cough, chest pain, breathing difficulty, low oxygen saturation, pleural effusion, cyanosis, and increased heart rate. However, it is important to note that some individuals may have no signs or symptoms. It is a common misconception that atelectasis causes fever, but there is no clinical evidence supporting this speculation. Atelectasis can be diagnosed through chest X-ray, which may show lung opacification and/or loss of lung volume. In some cases, additional tests such as chest CT or bronchoscopy may be necessary to determine the cause of atelectasis.\n',
 'sources': 'https://en.wikipedia.org/wiki/Atelectasis'}

In [9]:
chain(
    {"question": "What can cause Atelectasis?"},
    return_only_outputs=True,
)

{'answer': 'Atelectasis can be caused by various medical conditions, post-operative complications, surfactant deficiency, and restricted breathing after abdominal surgery. Other causes include pulmonary tuberculosis, blockage of a bronchiole or bronchus, poor surfactant spreading, and disruption of contact between the parietal and visceral pleurae. Risk factors for atelectasis include certain types of surgery, use of muscle relaxation, obesity, high oxygen, lower lung segments, age, presence of chronic obstructive pulmonary disease or asthma, and type of anesthetic. Diagnosis is typically done through chest X-ray, but additional imaging or bronchoscopy may be necessary in some cases.\n',
 'sources': 'https://en.wikipedia.org/wiki/Atelectasis'}

In [10]:
chain(
    {"question": "What clinical features radiologists need for diagnosing Atelectasis?"},
    return_only_outputs=True,
)

{'answer': 'Radiologists need to look for signs and symptoms such as cough, chest pain, breathing difficulty, low oxygen saturation, pleural effusion, cyanosis, and increased heart rate for diagnosing Atelectasis. Clinically significant atelectasis is generally visible on chest X-ray, which may show lung opacification and/or loss of lung volume. Additional diagnostic tools such as chest CT or bronchoscopy may be necessary if the cause of atelectasis is not clinically apparent.\n',
 'sources': 'https://en.wikipedia.org/wiki/Atelectasis'}

In [11]:
chain(
    {"question": "what are radiographic features of Atelectasis?"},
    return_only_outputs=True,
)

{'answer': 'The radiographic features of atelectasis include lung opacification and/or loss of lung volume, as visible on a chest X-ray. Post-surgical atelectasis typically presents as bibasal in pattern. Additional diagnostic tools such as chest CT or bronchoscopy may be necessary if the cause of atelectasis is not clinically apparent.\n',
 'sources': 'https://en.wikipedia.org/wiki/Atelectasis'}